# Gymnasium, SB3 실습

목표:
- Gymnasium 환경 생성 및 설정
- SB3 agent 생성 및 학습
- agent 성능 시각화

## 1. 라이브러리 import

In [4]:
# 라이브러리 import
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO  # SB3 알고리즘 중 하나
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback


## 2. 환경 생성

In [5]:
env_name = "CartPole-v1"
env = gym.make(env_name)


## 3. 환경 시각화

In [7]:
observation, info = env.reset()

for _ in range(10):
    env.render()
    action = env.action_space.sample()  # 랜덤한 행동 선택
    observation, reward, done, truncated, info = env.step(action)
    
    # `done` 또는 `truncated` 중 하나라도 True이면 에피소드가 끝난 것
    if done or truncated:
        observation, info = env.reset()

env.close()


## 4. Random Agent로 환경 테스트

In [9]:
for episode in range(5):  # 5번의 에피소드 실행
    observation, info = env.reset()  # info도 함께 받음
    total_reward = 0

    while True:
        action = env.action_space.sample()  # 랜덤 행동
        observation, reward, done, truncated, info = env.step(action)  # truncated 추가
        total_reward += reward
        
        if done or truncated:  # done 또는 truncated가 True이면 에피소드 종료
            print(f"Episode: {episode + 1}, Total Reward: {total_reward}")
            break

env.close()



Episode: 1, Total Reward: 29.0
Episode: 2, Total Reward: 18.0
Episode: 3, Total Reward: 22.0
Episode: 4, Total Reward: 16.0
Episode: 5, Total Reward: 17.0


## 5. 모델 생성 및 학습

- EvalCallback을 사용하여 학습 과정 log
- tensorboard로 학습 과정 시각화

In [10]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

# 환경 생성
env = make_vec_env("CartPole-v1", n_envs=4)  # 병렬 환경을 생성

# 모델 생성 (PPO 사용)
model = PPO("MlpPolicy", env, verbose=1)

# EvalCallback을 사용하여 학습 중에 모델을 평가
eval_env = Monitor(gym.make("CartPole-v1"))  # 평가 환경을 모니터링
eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=1000,
                             deterministic=True, render=False)

# 학습 시작
model.learn(total_timesteps=10000, callback=eval_callback)

# 학습 완료 후 모델 저장
model.save("ppo_cartpole")

# tensorboard로 학습 과정 시각화
# 학습 중에 터미널에서 다음 명령어를 실행하여 TensorBoard를 확인하세요.
# tensorboard --logdir ./logs/


Using cpu device
Eval num_timesteps=4000, episode_reward=149.40 +/- 27.41
Episode length: 149.40 +/- 27.41
---------------------------------
| eval/              |          |
|    mean_ep_length  | 149      |
|    mean_reward     | 149      |
| time/              |          |
|    total_timesteps | 4000     |
---------------------------------
New best mean reward!
Eval num_timesteps=8000, episode_reward=152.40 +/- 25.88
Episode length: 152.40 +/- 25.88
---------------------------------
| eval/              |          |
|    mean_ep_length  | 152      |
|    mean_reward     | 152      |
| time/              |          |
|    total_timesteps | 8000     |
---------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.2     |
|    ep_rew_mean     | 23.2     |
| time/              |          |
|    fps             | 1572     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_

## 6. 학습된 모델 시각화

In [12]:
import matplotlib.pyplot as plt
import os

# 학습된 모델 로드
model = PPO.load("ppo_cartpole")

# 학습된 모델로 시각화
env = gym.make("CartPole-v1", render_mode="rgb_array")
observation, info = env.reset()
frames = []  # 프레임 저장 리스트

for _ in range(1000):
    action, _states = model.predict(observation, deterministic=True)
    observation, reward, done, truncated, info = env.step(action)
    
    # 각 프레임을 저장 (이미지 시퀀스로 비디오 생성 가능)
    frame = env.render()
    frames.append(frame)

    if done or truncated:
        observation, info = env.reset()

env.close()

# MoviePy를 사용하여 비디오 저장
import moviepy.editor as mpy

video_dir = './videos'
os.makedirs(video_dir, exist_ok=True)
clip = mpy.ImageSequenceClip(frames, fps=30)
clip.write_videofile(f"{video_dir}/cartpole.mp4")


Moviepy - Building video ./videos/cartpole.mp4.
Moviepy - Writing video ./videos/cartpole.mp4



Moviepy - Done !
Moviepy - video ready ./videos/cartpole.mp4


## 7. 더 탐구해보기

- 어떤 모델과 환경이 호환되지 않는 이유가 무엇일까요?
- 학습된 모델의 성능을 높이기 위해 어떤 방법을 사용할 수 있을까요?
- 여러 환경과 모델을 사용하여 학습을 진행하고, 성능을 비교해보세요.
- 다양한 Wrappers, Callbacks를 활용하여 원하는 동작을 구현해보세요.